In [ ]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LSTM
from keras.layers import  Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import datetime
from datetime import time
import numpy as np

def calculate_mape(actual, predicted):
    """
    计算 MAPE（平均绝对百分比误差）

    参数:
    
    
    actual (array-like): 实际值数组。
    predicted (array-like): 预测值数组。

    返回:
    float: MAPE 值。
    """
    # 将输入转换为 numpy 数组
    actual = np.array(actual)
    predicted = np.array(predicted)

    # 避免除以零的情况
    if np.any(actual == 0):
        raise ValueError("实际值中包含零，无法计算 MAPE。")

    # 计算 MAPE
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    return mape

#for month_number in ['2024-11-15']:
for month_number in ['2025-1-15','2025-2-15','2025-3-15']:
    #ETL Process
    data = pd.read_excel('CS South WFM STAT 2025_05_14.xlsx')
    pd.set_option('display.max_rows', 100)
    data.columns = ['conversation_start_interval_tmst', 'Time', 'offered', 'actans',
           'actabn', 'absActHt', 'absActSa', 'parent', 'child', 'fiscalDate',
           'fiscalYear', 'ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW']
    
    data = data[~data .offered.isna()]
    
    data = data[data .offered != 0]
    # Month one and half need to set up 
    data_=data
    data = data[data.conversation_start_interval_tmst<=pd.to_datetime('{} 00:00:00'.format(month_number))]
    holiday_data = pd.read_excel(r'holiday.xlsx')
    holiday_data['~is_holiday'] = 0
    Holiday_name = ['Christmas Day', 'Columbus Day',
           'Independence Day', 'Labor Day', 'Martin Luther King Jr. Day',
           'Memorial Day', "New Year's Day", "Presidents' Day", 'Thanksgiving Day',
           'Veterans Day']
    holiday_data['date'] = holiday_data.Date.dt.date
    data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
    data['date'] = data.datetime.dt.date
    
    data.datetime = data.datetime.dt.floor('T')
    data['datetime'] = data['datetime'].apply(
        lambda x: x.ceil('30T') if x.minute == 29 else x
    )
    data['datetime'] = data['datetime'].apply(
        lambda x: x.ceil('H') if x.minute == 59 else x
    )
    data['time'] = data.datetime.dt.time
    data = data.sort_values('datetime').reset_index(drop = True)
    data = data.sort_values(by='datetime')
    
    # 获取所有唯一的时间点（如每天的09:00, 09:30等）
    unique_times = data['time'].unique()
    
    # 获取所有唯一的日期
    unique_dates = data['date'].unique()
    data = pd.merge(data,holiday_data,on = 'date',how = 'left').fillna(1)
    data = data[data['~is_holiday'] == 1]
    
    holiday_datetime = holiday_data.date.to_numpy()
    
    start_time = 35+23 # 58 how many days in advance
    end_time = start_time+36 #36 is the train length
    # set up the interval need to forecast here
    unique_times = [ datetime.time(9, 0), datetime.time(9, 30), datetime.time(10, 0),
    datetime.time(10, 30), datetime.time(11, 0), datetime.time(11, 30),
    datetime.time(12, 0), datetime.time(12, 30), datetime.time(13, 0),
    datetime.time(13, 30), datetime.time(14, 0), datetime.time(14, 30),
    datetime.time(15, 0), datetime.time(15, 30), datetime.time(16, 0),
    datetime.time(16, 30), datetime.time(17, 0), datetime.time(17, 30)
    ]

    
    add_column = ['actans','actabn','absActHt','absActSa']
    
    add_columns =  [f'actans_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'actabn_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'absActHt_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'absActSa_{i}' for i in range(start_time+1,end_time+1) ]
    
    matrix = []
    
    # 遍历每个时间点
    for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
        # 过滤出当前时间点的数据
        time_data = data[data['time'] == time].set_index('date')['offered']    
        full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max(), freq='B')  # 仅工作日
        full_dates = full_dates.difference(holiday_datetime)
        time_data = time_data.groupby(time_data.index).sum()
        time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        add_data = data[data['time'] == time].set_index('date')[add_column]
        add_data = add_data.groupby(add_data.index).sum()
        add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        time_data1 = data[data['time'] == time1].set_index('date')['offered']    
        time_data1 = time_data1.groupby(time_data1.index).sum()
        time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)
        
        time_data2 = data[data['time'] == time2].set_index('date')['offered']    
        time_data2 = time_data2.groupby(time_data2.index).sum()
        time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        year_data = time_data.shift(251)
        
    
        # 将当前时间点、前半小时和后半小时的数据拼接
        combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)
        
        # 创建日期+时间列
        datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)
        
        # 将日期+时间列添加到数据中
        combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])
        
        # 将结果存入矩阵
        matrix.append(combined_data_with_datetime)
    
    max_rows = max(arr.shape[0] for arr in matrix)
    
    # 将每个时间点的数据填充到最大行数
    for i in range(len(matrix)):
        if matrix[i].shape[0] < max_rows:
            padding = np.full((max_rows - matrix[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
            matrix[i] = np.vstack([matrix[i], padding])
    
    # 将矩阵堆叠成一个大的二维数组
    matrix = np.vstack(matrix)
    
    matrix = pd.DataFrame(matrix,columns = ['datetime','offered'] +add_columns +  [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                          + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + \
                          [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                           ['years_data']).sort_values('datetime').reset_index(drop = True)
    
    matrix['year'] = matrix.datetime.map(lambda x:x.year)
    matrix['month'] = matrix.datetime.map(lambda x:x.month)
    matrix['day'] = matrix.datetime.map(lambda x:x.day)
    matrix['hour'] = matrix.datetime.map(lambda x:x.hour)
    matrix['minute'] = matrix.datetime.map(lambda x:x.minute)
    
    matrix = matrix.sort_values('datetime').reset_index(drop = True)
    
    read_data = matrix.iloc[:]
    read_data = read_data[~read_data.offered.isna()].reset_index(drop = True)
    
    fill_na = read_data.iloc[:,1:].apply(lambda x:pd.to_numeric(x))
    
    
    
    fill_na = fill_na.interpolate(method='linear')
    
    fill_na['datetime'] = read_data.datetime
    fill_na['date'] = fill_na.datetime.dt.date
    fill_na = fill_na.dropna().reset_index(drop = True)
    
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['month'],prefix = 'month').astype(int)],axis = 1)
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['year'],prefix = 'year').astype(int)],axis = 1)
    fill_na['week_of_year'] = fill_na['datetime'].dt.isocalendar().week
    fill_na['weekday'] = fill_na['datetime'].dt.day_name()
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['weekday'],prefix = 'weekday').astype(int)],axis = 1)
    
    matrix_predict = []
    
    # 遍历每个时间点
    for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
        # 过滤出当前时间点的数据
        time_data = data[data['time'] == time].set_index('date')['offered']    
        full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max()+ pd.offsets.BDay(start_time), freq='B')  # 仅工作日
        full_dates = full_dates.difference(holiday_datetime)
        time_data = time_data.groupby(time_data.index).sum()    
        time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        add_data = data[data['time'] == time].set_index('date')[add_column]
        add_data = add_data.groupby(add_data.index).sum()
        add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        
        time_data1 = data[data['time'] == time1].set_index('date')['offered'] 
        time_data1 = time_data1.groupby(time_data1.index).sum()
        time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)
        
        time_data2 = data[data['time'] == time2].set_index('date')['offered']  
        time_data2 = time_data2.groupby(time_data2.index).sum()
        time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        year_data = time_data.shift(251)
        
    
        # 将当前时间点、前半小时和后半小时的数据拼接
        combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)
        
        # 创建日期+时间列
        datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)
        
        # 将日期+时间列添加到数据中
        combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])[-start_time:]
        
        
        
        # 将结果存入矩阵
        matrix_predict.append(combined_data_with_datetime)
    
    max_rows = max(arr.shape[0] for arr in matrix_predict)
    # 将每个时间点的数据填充到最大行数
    for i in range(len(matrix_predict)):
        if matrix_predict[i].shape[0] < max_rows:
            padding = np.full((max_rows - matrix_predict[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
            matrix_predict[i] = np.vstack([matrix_predict[i], padding])
    
    # 将矩阵堆叠成一个大的二维数组
    matrix_predict = np.vstack(matrix_predict)
    
    matrix_predict = pd.DataFrame(matrix_predict,columns = ['datetime','offered'] + add_columns+ [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                          + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                           ['years_data']).sort_values('datetime').reset_index(drop = True)
    
    matrix_predict['year'] = matrix_predict.datetime.map(lambda x:x.year)
    matrix_predict['month'] = matrix_predict.datetime.map(lambda x:x.month)
    matrix_predict['day'] = matrix_predict.datetime.map(lambda x:x.day)
    matrix_predict['hour'] = matrix_predict.datetime.map(lambda x:x.hour)
    matrix_predict['minute'] = matrix_predict.datetime.map(lambda x:x.minute)
    
    matrix_predict = matrix_predict.sort_values('datetime').reset_index(drop = True)
    
    read_data_predict = matrix_predict.iloc[:]
    
    
    
    
    fill_na_predict = read_data_predict.iloc[:,2:].apply(lambda x:pd.to_numeric(x))
    
    fill_na_predict = fill_na_predict.interpolate(method='linear')
    fill_na_predict = fill_na_predict.fillna(1)
    
    fill_na_predict['datetime'] = read_data_predict.datetime
    fill_na_predict['date'] = fill_na_predict.datetime.dt.date
    
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['month'],prefix = 'month').astype(int)],axis = 1)
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['year'], prefix = 'year').astype(int)],axis = 1)
    fill_na_predict['week_of_year'] = fill_na_predict['datetime'].dt.isocalendar().week
    fill_na_predict['weekday'] = fill_na_predict['datetime'].dt.day_name()
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['weekday'], prefix = 'weekday').astype(int)],axis = 1)
    
    fill_na_predict
    
    X_columns = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data'] + [f'month_{i}' for i in range(1,13)] +\
                [f'year_{i}' for i in np.unique(fill_na.year)] +\
                [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                    add_columns
                
    X_columns2 = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data'] + [f'month_{i}' for i in range(1,13)] +\
                [f'year_{i}' for i in np.unique(fill_na.year)] +\
                [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                    add_columns
    
    
    lstm_data = fill_na.dropna(axis = 0)
    
    lstm_data
    
    for i in X_columns:
        try:
            fill_na_predict[i]
        except:
            fill_na_predict[i] = 0
    
    lstm_data = lstm_data[lstm_data.year>=2021].reset_index(drop = True)
    
    week_dict = {'Friday':5, 'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4}
    
    lstm_data['week_num'] = lstm_data['weekday'].map(lambda x:week_dict[x])
    fill_na_predict['week_num'] = fill_na_predict['weekday'].map(lambda x:week_dict[x])
    all_feature = pd.DataFrame()
    #for week_ in [1,2,3,4,5]:
    for (hour,minute) in lstm_data[['hour','minute']].drop_duplicates().sort_values(['hour','minute']).to_numpy()[:]:
        now_lstm_data = lstm_data[(lstm_data.hour == hour)&(lstm_data.minute == minute)]
        #now_lstm_data = now_lstm_data[now_lstm_data.week_num == week_]
        now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
        #now_fill_na_predict = now_fill_na_predict[now_fill_na_predict.week_num == week_]
    
        now_time_list = now_lstm_data.datetime.to_numpy()
        now_lstm_data = now_lstm_data[['offered']+['datetime'] +X_columns].reset_index(drop = True)
        # 归一化数据
    
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-start_time])
        # 设置时间步长
        time_step = start_time
        # 创建训练数据集
        X_train_1, y_train =now_lstm_data[X_columns][:-start_time] , scaled_data
    
        X_test_1, y_test =now_lstm_data[X_columns][-start_time:] , now_lstm_data[['offered']][-start_time:]
    
        X_train_1 = np.array(X_train_1)
        y_train = np.array(y_train)
        X_test_1 = np.array(X_test_1)
        y_test = np.array(y_test)
        X_test_1 = X_test_1.reshape(X_test_1.shape[0], X_test_1.shape[1], 1)
    
        # 重塑输入数据为 [samples, time steps, features] 格式
        X_train_1 = X_train_1.reshape(X_train_1.shape[0], X_train_1.shape[1], 1)
        # 构建LSTM模型
        model_1 = Sequential()
        model_1.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model_1.add(LSTM(50, return_sequences=False))
        model_1.add(Dense(25))
        model_1.add(Dense(1))
    
        # 编译模型
        model_1.compile(optimizer='adam', loss='mean_squared_error')
        model_1.fit(X_train_1, y_train, batch_size=32, epochs=20)
        train_predict_1 = model_1.predict(X_train_1)
        train_predict_1 = scaler.inverse_transform(train_predict_1)
        True_data_1 = scaler.inverse_transform(y_train.reshape(-1, 1))
        print(calculate_mape(train_predict_1, True_data_1))
        return_y_1 =  model_1.predict(X_test_1)
        return_y_pred_1 = scaler.inverse_transform(np.array(return_y_1).reshape(-1, 1))
        now_predict_1 = pd.DataFrame(return_y_pred_1,columns = ['predict'])
        now_predict_1['datetime'] = now_time_list[-start_time:]
        now_lstm_data = now_lstm_data[['offered'] +X_columns2].reset_index(drop = True)
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-start_time])
        # 设置时间步长
        time_step = start_time
        # 创建训练数据集
    
        X_train_2, y_train =now_lstm_data[X_columns2][:-start_time] , scaled_data
    
        X_test_2, y_test =now_lstm_data[X_columns2][-start_time:] , now_lstm_data[['offered']][-start_time:]
    
        X_train_2 = np.array(X_train_2)
        y_train = np.array(y_train)
        X_test_2 = np.array(X_test_2)
        y_test = np.array(y_test)
        X_test_2 = X_test_2.reshape(X_test_2.shape[0], X_test_2.shape[1], 1)
    
        # 重塑输入数据为 [samples, time steps, features] 格式
        X_train_2 = X_train_2.reshape(X_train_2.shape[0], X_train_2.shape[1], 1)
        # 构建LSTM模型
        model_2 = Sequential()
        model_2.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model_2.add(LSTM(50, return_sequences=False))
        model_2.add(Dense(25))
        model_2.add(Dense(1))
    
        # 编译模型
        model_2.compile(optimizer='adam', loss='mean_squared_error')
        model_2.fit(X_train_2, y_train, batch_size=32, epochs=20)
        train_predict_2 = model_2.predict(X_train_2)
        train_predict_2 = scaler.inverse_transform(train_predict_2)
        True_data_2 = scaler.inverse_transform(y_train.reshape(-1, 1))
        print(calculate_mape(train_predict_2, True_data_2))
        return_y_2 =  model_2.predict(X_test_2)
        return_y_pred_2 = scaler.inverse_transform(np.array(return_y_2).reshape(-1, 1))
        now_predict_2 = pd.DataFrame(return_y_pred_2,columns = ['predict'])
        now_predict_2['datetime'] = now_time_list[-start_time:]
        print(calculate_mape(y_test, return_y_pred_2))
        all_predict=pd.DataFrame()
        if calculate_mape(y_test, return_y_pred_1) < calculate_mape(y_test, return_y_pred_2):
            all_predict = pd.concat([now_predict_1,all_predict])
            future_X = now_fill_na_predict[X_columns].to_numpy()
            future_data = scaler.inverse_transform(np.array(\
                            model_1.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
            future_data = pd.DataFrame(future_data,columns = ['predict'])
            all_feature = pd.concat([all_feature,future_data])
        else:
            all_predict = pd.concat([now_predict_2,all_predict])
            future_X = now_fill_na_predict[X_columns2].to_numpy()
            future_data = scaler.inverse_transform(np.array(\
                            model_2.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
            future_data = pd.DataFrame(future_data,columns = ['predict'])
            all_feature = pd.concat([all_feature,future_data])
        #print('Interval Hour:{}, Minute:{} has been trained'.format(hour,minute)
    all_predict = pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
    all_predict = all_predict.sort_values('datetime').reset_index(drop = True)
    #all_feature.to_excel(r'return_month_{}.xlsx'.format(month_number))
    all_predict['diff'] = (all_predict.predict - all_predict.offered)
    all_predict['hour'] = all_predict.datetime.dt.hour
    pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
    calculate_mape(all_predict.offered,all_predict.predict)
    all_predict.to_excel(r'Test_{}.xlsx'.format(month_number))
    time_ = []
    for (hour,minute) in fill_na_predict[['hour','minute']].drop_duplicates().to_numpy()[:]:
        now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
        time_.append(now_fill_na_predict.datetime.to_numpy())
    all_feature['datetime'] =  np.hstack(time_)
    all_feature.predict = all_feature.predict.map(lambda x:max(0,x))
    data_['conversation_start_interval_tmst']=pd.to_datetime(data_['conversation_start_interval_tmst'])
    all_feature['datetime']=pd.to_datetime(all_feature['datetime'])
    all_feature = pd.merge(all_feature,data_[['offered','conversation_start_interval_tmst']],left_on = ['datetime'],right_on=['conversation_start_interval_tmst'])
    all_feature = all_feature.sort_values('datetime').reset_index(drop = True)
    all_feature['mape']=np.abs(all_feature['offered']-all_feature['predict'])/(all_feature['offered'])*100
    all_feature.to_excel(r'Target_{}.xlsx'.format(month_number))

In [1]:
import pandas

ModuleNotFoundError: No module named 'pandas'

In [3]:
import numpy as np
np.

In [ ]:
np.

In [ ]:
pd. dataf

In [3]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LSTM
from keras.layers import  Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
data = pd.read_excel('RS_KSR_ACT_WFM_STAT.xlsx')
pd.set_option('display.max_rows', 100)

2025-05-14 07:00:36.948159: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-14 07:00:36.951403: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-14 07:00:36.961624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747206036.978592    1726 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747206036.983724    1726 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747206036.996360    1726 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [4]:
data.columns = ['conversation_start_interval_tmst', 'Time', 'offered', 'actans',
       'actabn', 'absActHt', 'absActSa', 'parent', 'child', 'fiscalDate',
       'fiscalYear', 'ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW']

In [5]:
data = data[~data .offered.isna()]

In [6]:
data = data[data .offered != 0]

In [ ]:
import pandas as pd

In [2]:
!which python

/usr/bin/python


In [7]:
data = data[data['conversation_start_interval_tmst']<=pd.to_datetime('{} 00:00:00'.format('2024-11-15'))]

In [8]:
data['conversation_start_interval_tmst'].max()

Timestamp('2024-09-20 19:30:00')

In [110]:
# 假设数据有两列：'timestamp' 和 'value'
holiday_data = pd.read_excel(r'holiday.xlsx')
holiday_data['~is_holiday'] = 0
Holiday_name = ['Christmas Day', 'Columbus Day',
       'Independence Day', 'Labor Day', 'Martin Luther King Jr. Day',
       'Memorial Day', "New Year's Day", "Presidents' Day", 'Thanksgiving Day',
       'Veterans Day']
holiday_data['date'] = holiday_data.Date.dt.date
data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
data['date'] = data.datetime.dt.date

data.datetime = data.datetime.dt.floor('T')
data['datetime'] = data['datetime'].apply(
    lambda x: x.ceil('30T') if x.minute == 29 else x
)
data['datetime'] = data['datetime'].apply(
    lambda x: x.ceil('H') if x.minute == 59 else x
)
data['time'] = data.datetime.dt.time
data = data.sort_values('datetime').reset_index(drop = True)
data = data.sort_values(by='datetime')
data = data[data.fiscalDate<pd.to_datetime('2025-04-15 00:00:00')]
# 获取所有唯一的时间点（如每天的09:00, 09:30等）
unique_times = data['time'].unique()

# 获取所有唯一的日期
unique_dates = data['date'].unique()
data = pd.merge(data,holiday_data,on = 'date',how = 'left').fillna(1)
data = data[data['~is_holiday'] == 1]

C:\Users\nemo\AppData\Local\Temp\ipykernel_7388\776040365.py:12: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  data.datetime = data.datetime.dt.floor('T')


In [111]:
holiday_datetime = holiday_data.date.to_numpy()

In [112]:
from datetime import time

In [ ]:
import pandas as pd


In [113]:
unique_times = [
time(9, 0), time(9, 30), time(10, 0),
time(10, 30), time(11, 0), time(11, 30),
time(12, 0), time(12, 30), time(13, 0),
time(13, 30), time(14, 0), time(14, 30),
time(15, 0), time(15, 30), time(16, 0),
time(16, 30), time(17, 0), time(17, 30),
time(18, 0), time(18, 30), time(19, 0)]

In [114]:
import numpy as np

def calculate_mape(actual, predicted):
    """
    计算 MAPE（平均绝对百分比误差）

    参数:
    
    
    actual (array-like): 实际值数组。
    predicted (array-like): 预测值数组。

    返回:
    float: MAPE 值。
    """

    # 将输入转换为 numpy 数组
    actual = np.array(actual)
    predicted = np.array(predicted)

    # 避免除以零的情况
    if np.any(actual == 0):
        raise ValueError("实际值中包含零，无法计算 MAPE。")

    # 计算 MAPE
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    return mape


In [115]:
import pandas

NameError: name 'data' is not defined

In [ ]:
all_predict = pd.DataFrame()
all_feature = pd.DataFrame()

for now_day_pre,now_predict_date in enumerate(pd.date_range(start=unique_dates.max()+ pd.offsets.BDay(32), end=unique_dates.max()+ pd.offsets.BDay(34+23), freq='B')):
    start_time = 32+now_day_pre #用前35+n 到35+n+36天的数据预测当前时刻的数据
    end_time = start_time+36 #
    test_long = 70
    #test_long=0
    test_start = start_time+test_long
    test_end  = end_time+test_long
    
    f_predict = pd.DataFrame()
    add_column = ['actans','actabn','absActHt','absActSa']
    add_columns =  [f'actans_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'actabn_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'absActHt_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'absActSa_{i}' for i in range(start_time+1,end_time+1) ]

    
    matrix = []

    # 遍历每个时间点
    for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
        # 过滤出当前时间点的数据
        time_data = data[data['time'] == time].set_index('date')['offered']    
        full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max(), freq='B')  # 仅工作日
        full_dates = full_dates.difference(holiday_datetime)
        time_data = time_data.groupby(time_data.index).sum()
        time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)


        add_data = data[data['time'] == time].set_index('date')[add_column]
        add_data = add_data.groupby(add_data.index).sum()
        add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)


        time_data1 = data[data['time'] == time1].set_index('date')['offered']    
        time_data1 = time_data1.groupby(time_data1.index).sum()
        time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)

        time_data2 = data[data['time'] == time2].set_index('date')['offered']    
        time_data2 = time_data2.groupby(time_data2.index).sum()
        time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)


        year_data = time_data.shift(251)


        # 将当前时间点、前半小时和后半小时的数据拼接
        combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)

        # 创建日期+时间列
        datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)

        # 将日期+时间列添加到数据中
        combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])

        # 将结果存入矩阵
        matrix.append(combined_data_with_datetime)

    max_rows = max(arr.shape[0] for arr in matrix)

    # 将每个时间点的数据填充到最大行数
    for i in range(len(matrix)):
        if matrix[i].shape[0] < max_rows:
            padding = np.full((max_rows - matrix[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
            matrix[i] = np.vstack([matrix[i], padding])

    # 将矩阵堆叠成一个大的二维数组
    matrix = np.vstack(matrix)

    matrix = pd.DataFrame(matrix,columns = ['datetime','offered'] +add_columns +  [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                          + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + \
                          [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                           ['years_data']).sort_values('datetime').reset_index(drop = True)

    matrix['year'] = matrix.datetime.map(lambda x:x.year)
    matrix['month'] = matrix.datetime.map(lambda x:x.month)
    matrix['day'] = matrix.datetime.map(lambda x:x.day)
    matrix['hour'] = matrix.datetime.map(lambda x:x.hour)
    matrix['minute'] = matrix.datetime.map(lambda x:x.minute)

    matrix = matrix.sort_values('datetime').reset_index(drop = True)

    read_data = matrix.iloc[:]
    read_data = read_data[~read_data.offered.isna()].reset_index(drop = True)

    fill_na = read_data.iloc[:,1:].apply(lambda x:pd.to_numeric(x))

    fill_na = fill_na.interpolate(method='linear')

    fill_na['datetime'] = read_data.datetime
    fill_na['date'] = fill_na.datetime.dt.date
    fill_na = fill_na.dropna().reset_index(drop = True)

    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['month'],prefix = 'month').astype(int)],axis = 1)
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['year'],prefix = 'year').astype(int)],axis = 1)
    fill_na['week_of_year'] = fill_na['datetime'].dt.isocalendar().week
    fill_na['weekday'] = fill_na['datetime'].dt.day_name()
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['weekday'],prefix = 'weekday').astype(int)],axis = 1)

    matrix_predict = []




    # 遍历每个时间点
    for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
        # 过滤出当前时间点的数据
        time_data = data[data['time'] == time].set_index('date')['offered']    
        full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max()+ pd.offsets.BDay(start_time), freq='B')  # 仅工作日
        full_dates = full_dates.difference(holiday_datetime)
        time_data = time_data.groupby(time_data.index).sum()    
        time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)


        add_data = data[data['time'] == time].set_index('date')[add_column]
        add_data = add_data.groupby(add_data.index).sum()
        add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)



        time_data1 = data[data['time'] == time1].set_index('date')['offered'] 
        time_data1 = time_data1.groupby(time_data1.index).sum()
        time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)

        time_data2 = data[data['time'] == time2].set_index('date')['offered']  
        time_data2 = time_data2.groupby(time_data2.index).sum()
        time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)


        year_data = time_data.shift(251)


        # 将当前时间点、前半小时和后半小时的数据拼接
        combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)

        # 创建日期+时间列
        datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)

        # 将日期+时间列添加到数据中
        combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])[-1:]
        # 将结果存入矩阵
        matrix_predict.append(combined_data_with_datetime)

    max_rows = max(arr.shape[0] for arr in matrix_predict)
    # 将每个时间点的数据填充到最大行数
    for i in range(len(matrix_predict)):
        if matrix_predict[i].shape[0] < max_rows:
            padding = np.full((max_rows - matrix_predict[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
            matrix_predict[i] = np.vstack([matrix_predict[i], padding])

    # 将矩阵堆叠成一个大的二维数组
    matrix_predict = np.vstack(matrix_predict)

    matrix_predict = pd.DataFrame(matrix_predict,columns = ['datetime','offered'] + add_columns+ [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                          + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                           ['years_data']).sort_values('datetime').reset_index(drop = True)

    matrix_predict['year'] = matrix_predict.datetime.map(lambda x:x.year)
    matrix_predict['month'] = matrix_predict.datetime.map(lambda x:x.month)
    matrix_predict['day'] = matrix_predict.datetime.map(lambda x:x.day)
    matrix_predict['hour'] = matrix_predict.datetime.map(lambda x:x.hour)
    matrix_predict['minute'] = matrix_predict.datetime.map(lambda x:x.minute)

    matrix_predict = matrix_predict.sort_values('datetime').reset_index(drop = True)

    read_data_predict = matrix_predict.iloc[:]


    fill_na_predict = read_data_predict.iloc[:,2:].apply(lambda x:pd.to_numeric(x))

    fill_na_predict = fill_na_predict.interpolate(method='linear')
    fill_na_predict = fill_na_predict.fillna(1)

    fill_na_predict['datetime'] = read_data_predict.datetime
    fill_na_predict['date'] = now_predict_date

    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['month'],prefix = 'month').astype(int)],axis = 1)
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['year'], prefix = 'year').astype(int)],axis = 1)
    fill_na_predict['week_of_year'] = fill_na_predict['datetime'].dt.isocalendar().week
    fill_na_predict['weekday'] = fill_na_predict['datetime'].dt.day_name()
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['weekday'], prefix = 'weekday').astype(int)],axis = 1)
    X_columns = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data'] + [f'month_{i}' for i in range(1,13)] +\
                [f'year_{i}' for i in np.unique(fill_na.year)] +\
                [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                    add_columns

    X_columns2 = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data'] + [f'month_{i}' for i in range(1,13)] +\
                [f'year_{i}' for i in np.unique(fill_na.year)] +\
                [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                    add_columns


    lstm_data = fill_na.dropna(axis = 0)
    for i in X_columns:
        try:
            fill_na_predict[i]
        except:
            fill_na_predict[i] = 0
    lstm_data = lstm_data[lstm_data.year>=2021].reset_index(drop = True)
    week_dict = {'Friday':5, 'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4}
    lstm_data['week_num'] = lstm_data['weekday'].map(lambda x:week_dict[x])
    fill_na_predict['week_num'] = fill_na_predict['weekday'].map(lambda x:week_dict[x])


    #for week_ in [1,2,3,4,5]:

    for (hour,minute) in lstm_data[['hour','minute']].drop_duplicates().sort_values(['hour','minute']).to_numpy()[:]:
        now_lstm_data = lstm_data[(lstm_data.hour == hour)&(lstm_data.minute == minute)]
        #now_lstm_data = now_lstm_data[now_lstm_data.week_num == week_]
        now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
        #now_fill_na_predict = now_fill_na_predict[now_fill_na_predict.week_num == week_]

        now_time_list = now_lstm_data.datetime.to_numpy()
        now_lstm_data = now_lstm_data[['offered']+['datetime'] +X_columns].reset_index(drop = True)
        # 归一化数据

        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-test_start])
        # 设置时间步长
        time_step = start_time
        # 创建训练数据集
        X_train_1, y_train =now_lstm_data[X_columns][:-test_start] , scaled_data

        X_test_1, y_test =now_lstm_data[X_columns][-test_start:] , now_lstm_data[['offered']][-test_start:]

        X_train_1 = np.array(X_train_1)
        y_train = np.array(y_train)
        X_test_1 = np.array(X_test_1)
        y_test = np.array(y_test)
        X_test_1 = X_test_1.reshape(X_test_1.shape[0], X_test_1.shape[1], 1)

        # 重塑输入数据为 [samples, time steps, features] 格式
        X_train_1 = X_train_1.reshape(X_train_1.shape[0], X_train_1.shape[1], 1)
        # 构建LSTM模型
        model_1 = Sequential()
        model_1.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model_1.add(LSTM(50, return_sequences=False))
        model_1.add(Dense(25))
        model_1.add(Dense(1))

        # 编译模型
        model_1.compile(optimizer='adam', loss='mean_squared_error')
        model_1.fit(X_train_1, y_train, batch_size=32, epochs=20)
        train_predict_1 = model_1.predict(X_train_1)
        train_predict_1 = scaler.inverse_transform(train_predict_1)
        True_data_1 = scaler.inverse_transform(y_train.reshape(-1, 1))
        print(calculate_mape(train_predict_1, True_data_1))
        return_y_1 =  model_1.predict(X_test_1)
        return_y_pred_1 = scaler.inverse_transform(np.array(return_y_1).reshape(-1, 1))
        now_predict_1 = pd.DataFrame(return_y_pred_1,columns = ['predict'])
        now_predict_1['datetime'] = now_time_list[-test_start:]


        now_lstm_data = now_lstm_data[['offered'] +X_columns2].reset_index(drop = True)

        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-test_start])
        # 设置时间步长
        time_step = start_time
        # 创建训练数据集

        X_train_2, y_train =now_lstm_data[X_columns2][:-test_start] , scaled_data

        X_test_2, y_test =now_lstm_data[X_columns2][-test_start:] , now_lstm_data[['offered']][-test_start:]

        X_train_2 = np.array(X_train_2)
        y_train = np.array(y_train)
        X_test_2 = np.array(X_test_2)
        y_test = np.array(y_test)
        X_test_2 = X_test_2.reshape(X_test_2.shape[0], X_test_2.shape[1], 1)

        # 重塑输入数据为 [samples, time steps, features] 格式
        X_train_2 = X_train_2.reshape(X_train_2.shape[0], X_train_2.shape[1], 1)
        # 构建LSTM模型
        model_2 = Sequential()
        model_2.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model_2.add(LSTM(50, return_sequences=False))
        model_2.add(Dense(25))
        model_2.add(Dense(1))

        # 编译模型
        model_2.compile(optimizer='adam', loss='mean_squared_error')
        model_2.fit(X_train_2, y_train, batch_size=32, epochs=20)
        train_predict_2 = model_2.predict(X_train_2)
        train_predict_2 = scaler.inverse_transform(train_predict_2)
        True_data_2 = scaler.inverse_transform(y_train.reshape(-1, 1))
        print(calculate_mape(train_predict_2, True_data_2))
        return_y_2 =  model_2.predict(X_test_2)
        return_y_pred_2 = scaler.inverse_transform(np.array(return_y_2).reshape(-1, 1))
        now_predict_2 = pd.DataFrame(return_y_pred_2,columns = ['predict'])
        now_predict_2['datetime'] = now_time_list[-test_start:]
        print(calculate_mape(y_test, return_y_pred_2))
        if calculate_mape(y_test, return_y_pred_1) < calculate_mape(y_test, return_y_pred_2):


            f_predict = pd.concat([now_predict_1,f_predict])
            future_X = now_fill_na_predict[X_columns].to_numpy()
            future_data = scaler.inverse_transform(np.array(\
                            model_1.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
            future_data = pd.DataFrame(future_data,columns = ['predict'])
            future_data['day'] = now_predict_date
            future_data['hour'] = hour
            future_data['minute'] = minute
            all_feature = pd.concat([all_feature,future_data])
        else:
            f_predict = pd.concat([now_predict_2,f_predict])
            future_X = now_fill_na_predict[X_columns2].to_numpy()
            future_data = scaler.inverse_transform(np.array(\
                            model_2.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
            future_data = pd.DataFrame(future_data,columns = ['predict'])
            future_data['day'] = now_predict_date
            future_data['hour'] = hour
            future_data['minute'] = minute
            all_feature = pd.concat([all_feature,future_data])
    f_predict = pd.merge(f_predict,lstm_data[['offered','datetime']],on = ['datetime'])
    f_predict = f_predict.sort_values('datetime').reset_index(drop = True)
    all_predict = pd.concat([all_predict,f_predict])
    print(all_predict.head(2))

    #all_predict.to_excel(r'return2.xlsx')


Epoch 1/20


C:\Users\nemo\anaconda3\envs\lstm_env\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 274ms/step - loss: 0.0383
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 314ms/step - loss: 0.0217
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 278ms/step - loss: 0.0207
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 271ms/step - loss: 0.0279
Epoch 5/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 282ms/step - loss: 0.0190
Epoch 6/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 269ms/step - loss: 0.0185
Epoch 7/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 277ms/step - loss: 0.0169
Epoch 8/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 267ms/step - loss: 0.0159
Epoch 9/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 273ms/step - loss: 0.0196
Epoch 10/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 269ms/step - loss: 0.0181
Epoch 11/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 275ms/step - loss: 0.0147
Epoch 12/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 277ms/step - loss: 0.0165
Epoch 13/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 275ms/step - loss: 0.0208
Epoch 14/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 276ms/step - loss: 0.0176
Epoch 15/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 275ms/step - loss: 0.0161


C:\Users\nemo\anaconda3\envs\lstm_env\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 272ms/step - loss: 0.1634
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 276ms/step - loss: 0.0386
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 278ms/step - loss: 0.0233
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 279ms/step - loss: 0.0206
Epoch 5/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 269ms/step - loss: 0.0192
Epoch 6/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 272ms/step - loss: 0.0203
Epoch 7/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 277ms/step - loss: 0.0189
Epoch 8/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 276ms/step - loss: 0.0176
Epoch 9/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 264ms/step - loss: 0.0177
Epoch 10/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 258ms/step - loss: 0.0199
Epoch 11/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 269ms/step - loss: 0.0197
Epoch 12/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 265ms/step - loss: 0.0223
Epoch 13/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 265ms/step - loss: 0.0189
Epoch 14/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 267ms/step - loss: 0.0184
Epoch 15/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 272ms/step - loss: 0.0182


...

13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 295ms/step - loss: 0.0613
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 298ms/step - loss: 0.0213
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 291ms/step - loss: 0.0195
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 302ms/step - loss: 0.0187
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 291ms/step - loss: 0.0176
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 276ms/step - loss: 0.0195
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 300ms/step - loss: 0.0197
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 295ms/step - loss: 0.0240
Epoch 9/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 289ms/step - loss: 0.0183
Epoch 10/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 289ms/step - loss: 0.0189
Epoch 11/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 295ms/step - loss: 0.0182
Epoch 12/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 281ms/step - loss: 0.0209
Epoch 13/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 277ms/step - loss: 0.0201
Epoch 14/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 283ms/step - loss: 0.0187
Epoch 15/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 305ms/step - loss: 0.0203


C:\Users\nemo\anaconda3\envs\lstm_env\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 289ms/step - loss: 0.0651
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 279ms/step - loss: 0.0252
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 291ms/step - loss: 0.0158
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 309ms/step - loss: 0.0164
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 293ms/step - loss: 0.0150
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 276ms/step - loss: 0.0152
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 297ms/step - loss: 0.0197
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 285ms/step - loss: 0.0174
Epoch 9/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 282ms/step - loss: 0.0160
Epoch 10/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 294ms/step - loss: 0.0154
Epoch 11/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 295ms/step - loss: 0.0169
Epoch 12/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 283ms/step - loss: 0.0170
Epoch 13/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 307ms/step - loss: 0.0182
Epoch 14/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 297ms/step - loss: 0.0152
Epoch 15/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 282ms/step - loss: 0.0154


C:\Users\nemo\anaconda3\envs\lstm_env\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 286ms/step - loss: 0.1058
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 290ms/step - loss: 0.0210
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 275ms/step - loss: 0.0181
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 299ms/step - loss: 0.0181
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 286ms/step - loss: 0.0170
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 280ms/step - loss: 0.0180
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 302ms/step - loss: 0.0169
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 296ms/step - loss: 0.0153
Epoch 9/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 275ms/step - loss: 0.0161
Epoch 10/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 286ms/step - loss: 0.0164
Epoch 11/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 297ms/step - loss: 0.0158
Epoch 12/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 285ms/step - loss: 0.0185
Epoch 13/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 280ms/step - loss: 0.0175
Epoch 14/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 299ms/step - loss: 0.0140
Epoch 15/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 292ms/step - loss: 0.0155


In [151]:
now_predict_1

,predict,datetime
0,18.063202,2024-06-06 19:00:00
1,18.382673,2024-06-07 19:00:00
2,18.245770,2024-06-10 19:00:00
3,17.934334,2024-06-11 19:00:00
4,17.662100,2024-06-12 19:00:00
...,...,...
158,12.959219,2025-04-08 19:00:00
159,12.202262,2025-04-09 19:00:00
160,14.455463,2025-04-10 19:00:00
161,16.271149,2025-04-11 19:00:00


In [150]:
pd.merge(f_predict,lstm_data[['offered','datetime']],on = ['datetime'])

,predict,datetime,offered_x,offered_y
0,55.071297,2024-06-05 11:00:00,48.0,48.0
1,53.964642,2024-06-05 12:00:00,52.0,52.0
2,59.808586,2024-06-05 12:30:00,67.0,67.0
3,56.079811,2024-06-05 13:00:00,43.0,43.0
4,53.681915,2024-06-05 14:00:00,68.0,68.0
...,...,...,...,...
3418,34.340652,2025-04-14 17:00:00,41.0,41.0
3419,32.661415,2025-04-14 17:30:00,41.0,41.0
3420,26.716602,2025-04-14 18:00:00,38.0,38.0
3421,23.531254,2025-04-14 18:30:00,33.0,33.0


In [146]:
all_predict = all_predict.groupby('datetime').mean()

In [147]:
((all_predict.offered - all_predict.predict)**2).mean()**0.5

24.133531663430198

In [148]:
calculate_mape(all_predict.offered,all_predict.predict)

45.205149277674245

In [54]:
for now_day_pre in range(23):
    start_time = 35+now_day_pre #用前35+n 到35+n+36天的数据预测当前时刻的数据
    end_time = start_time+36 #

    matrix = []

    # 遍历每个时间点
    for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
        # 过滤出当前时间点的数据
        time_data = data[data['time'] == time].set_index('date')['offered']    
        full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max(), freq='B')  # 仅工作日
        full_dates = full_dates.difference(holiday_datetime)
        time_data = time_data.groupby(time_data.index).sum()
        time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)


        add_data = data[data['time'] == time].set_index('date')[add_column]
        add_data = add_data.groupby(add_data.index).sum()
        add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)


        time_data1 = data[data['time'] == time1].set_index('date')['offered']    
        time_data1 = time_data1.groupby(time_data1.index).sum()
        time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)

        time_data2 = data[data['time'] == time2].set_index('date')['offered']    
        time_data2 = time_data2.groupby(time_data2.index).sum()
        time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)


        year_data = time_data.shift(251)


        # 将当前时间点、前半小时和后半小时的数据拼接
        combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)

        # 创建日期+时间列
        datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)

        # 将日期+时间列添加到数据中
        combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])

        # 将结果存入矩阵
        matrix.append(combined_data_with_datetime)

    max_rows = max(arr.shape[0] for arr in matrix)

    # 将每个时间点的数据填充到最大行数
    for i in range(len(matrix)):
        if matrix[i].shape[0] < max_rows:
            padding = np.full((max_rows - matrix[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
            matrix[i] = np.vstack([matrix[i], padding])

    # 将矩阵堆叠成一个大的二维数组
    matrix = np.vstack(matrix)

    matrix = pd.DataFrame(matrix,columns = ['datetime','offered'] +add_columns +  [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                          + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + \
                          [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                           ['years_data']).sort_values('datetime').reset_index(drop = True)

    matrix['year'] = matrix.datetime.map(lambda x:x.year)
    matrix['month'] = matrix.datetime.map(lambda x:x.month)
    matrix['day'] = matrix.datetime.map(lambda x:x.day)
    matrix['hour'] = matrix.datetime.map(lambda x:x.hour)
    matrix['minute'] = matrix.datetime.map(lambda x:x.minute)

    matrix = matrix.sort_values('datetime').reset_index(drop = True)

    read_data = matrix.iloc[:]
    read_data = read_data[~read_data.offered.isna()].reset_index(drop = True)

    fill_na = read_data.iloc[:,1:].apply(lambda x:pd.to_numeric(x))

    fill_na = fill_na.interpolate(method='linear')

    fill_na['datetime'] = read_data.datetime
    fill_na['date'] = fill_na.datetime.dt.date
    fill_na = fill_na.dropna().reset_index(drop = True)

    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['month'],prefix = 'month').astype(int)],axis = 1)
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['year'],prefix = 'year').astype(int)],axis = 1)
    fill_na['week_of_year'] = fill_na['datetime'].dt.isocalendar().week
    fill_na['weekday'] = fill_na['datetime'].dt.day_name()
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['weekday'],prefix = 'weekday').astype(int)],axis = 1)

    matrix_predict = []




    # 遍历每个时间点
    for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
        # 过滤出当前时间点的数据
        time_data = data[data['time'] == time].set_index('date')['offered']    
        full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max()+ pd.offsets.BDay(start_time), freq='B')  # 仅工作日
        full_dates = full_dates.difference(holiday_datetime)
        time_data = time_data.groupby(time_data.index).sum()    
        time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)


        add_data = data[data['time'] == time].set_index('date')[add_column]
        add_data = add_data.groupby(add_data.index).sum()
        add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)



        time_data1 = data[data['time'] == time1].set_index('date')['offered'] 
        time_data1 = time_data1.groupby(time_data1.index).sum()
        time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)

        time_data2 = data[data['time'] == time2].set_index('date')['offered']  
        time_data2 = time_data2.groupby(time_data2.index).sum()
        time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)


        year_data = time_data.shift(251)


        # 将当前时间点、前半小时和后半小时的数据拼接
        combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)

        # 创建日期+时间列
        datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)

        # 将日期+时间列添加到数据中
        combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])[-1:]
        # 将结果存入矩阵
        matrix_predict.append(combined_data_with_datetime)

    max_rows = max(arr.shape[0] for arr in matrix_predict)
    # 将每个时间点的数据填充到最大行数
    for i in range(len(matrix_predict)):
        if matrix_predict[i].shape[0] < max_rows:
            padding = np.full((max_rows - matrix_predict[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
            matrix_predict[i] = np.vstack([matrix_predict[i], padding])

    # 将矩阵堆叠成一个大的二维数组
    matrix_predict = np.vstack(matrix_predict)

    matrix_predict = pd.DataFrame(matrix_predict,columns = ['datetime','offered'] + add_columns+ [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                          + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                           ['years_data']).sort_values('datetime').reset_index(drop = True)

    matrix_predict['year'] = matrix_predict.datetime.map(lambda x:x.year)
    matrix_predict['month'] = matrix_predict.datetime.map(lambda x:x.month)
    matrix_predict['day'] = matrix_predict.datetime.map(lambda x:x.day)
    matrix_predict['hour'] = matrix_predict.datetime.map(lambda x:x.hour)
    matrix_predict['minute'] = matrix_predict.datetime.map(lambda x:x.minute)

    matrix_predict = matrix_predict.sort_values('datetime').reset_index(drop = True)

    read_data_predict = matrix_predict.iloc[:]


    fill_na_predict = read_data_predict.iloc[:,2:].apply(lambda x:pd.to_numeric(x))

    fill_na_predict = fill_na_predict.interpolate(method='linear')
    fill_na_predict = fill_na_predict.fillna(1)

    fill_na_predict['datetime'] = read_data_predict.datetime
    fill_na_predict['date'] = fill_na_predict.datetime.dt.date

    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['month'],prefix = 'month').astype(int)],axis = 1)
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['year'], prefix = 'year').astype(int)],axis = 1)
    fill_na_predict['week_of_year'] = fill_na_predict['datetime'].dt.isocalendar().week
    fill_na_predict['weekday'] = fill_na_predict['datetime'].dt.day_name()
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['weekday'], prefix = 'weekday').astype(int)],axis = 1)
    X_columns = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data'] + [f'month_{i}' for i in range(1,13)] +\
                [f'year_{i}' for i in np.unique(fill_na.year)] +\
                [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                    add_columns

    X_columns2 = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data'] + [f'month_{i}' for i in range(1,13)] +\
                [f'year_{i}' for i in np.unique(fill_na.year)] +\
                [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                    add_columns


    lstm_data = fill_na.dropna(axis = 0)
    for i in X_columns:
        try:
            fill_na_predict[i]
        except:
            fill_na_predict[i] = 0
    lstm_data = lstm_data[lstm_data.year>=2021].reset_index(drop = True)
    week_dict = {'Friday':5, 'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4}
    lstm_data['week_num'] = lstm_data['weekday'].map(lambda x:week_dict[x])
    fill_na_predict['week_num'] = fill_na_predict['weekday'].map(lambda x:week_dict[x])


    #for week_ in [1,2,3,4,5]:

    for (hour,minute) in lstm_data[['hour','minute']].drop_duplicates().sort_values(['hour','minute']).to_numpy()[:]:
        now_lstm_data = lstm_data[(lstm_data.hour == hour)&(lstm_data.minute == minute)]
        #now_lstm_data = now_lstm_data[now_lstm_data.week_num == week_]
        now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
        #now_fill_na_predict = now_fill_na_predict[now_fill_na_predict.week_num == week_]

        now_time_list = now_lstm_data.datetime.to_numpy()
        now_lstm_data = now_lstm_data[['offered']+['datetime'] +X_columns].reset_index(drop = True)
        # 归一化数据

        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-start_time])
        # 设置时间步长
        time_step = start_time
        # 创建训练数据集
        X_train_1, y_train =now_lstm_data[X_columns][:-start_time] , scaled_data

        X_test_1, y_test =now_lstm_data[X_columns][-start_time:-start_time+1] , now_lstm_data[['offered']][-start_time:-start_time+1]

        X_train_1 = np.array(X_train_1)
        y_train = np.array(y_train)
        X_test_1 = np.array(X_test_1)
        y_test = np.array(y_test)
        X_test_1 = X_test_1.reshape(X_test_1.shape[0], X_test_1.shape[1], 1)

        # 重塑输入数据为 [samples, time steps, features] 格式
        X_train_1 = X_train_1.reshape(X_train_1.shape[0], X_train_1.shape[1], 1)
        # 构建LSTM模型
        model_1 = Sequential()
        model_1.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model_1.add(LSTM(50, return_sequences=False))
        model_1.add(Dense(25))
        model_1.add(Dense(1))

        # 编译模型
        model_1.compile(optimizer='adam', loss='mean_squared_error')
        model_1.fit(X_train_1, y_train, batch_size=32, epochs=20)
        train_predict_1 = model_1.predict(X_train_1)
        train_predict_1 = scaler.inverse_transform(train_predict_1)
        True_data_1 = scaler.inverse_transform(y_train.reshape(-1, 1))
        print(calculate_mape(train_predict_1, True_data_1))
        return_y_1 =  model_1.predict(X_test_1)
        return_y_pred_1 = scaler.inverse_transform(np.array(return_y_1).reshape(-1, 1))
        now_predict_1 = pd.DataFrame(return_y_pred_1,columns = ['predict'])
        now_predict_1['datetime'] = now_time_list[-start_time:-start_time+1]


        now_lstm_data = now_lstm_data[['offered'] +X_columns2].reset_index(drop = True)

        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-start_time])
        # 设置时间步长
        time_step = start_time
        # 创建训练数据集

        X_train_2, y_train =now_lstm_data[X_columns2][:-start_time] , scaled_data

        X_test_2, y_test =now_lstm_data[X_columns2][-start_time:-start_time+1] , now_lstm_data[['offered']][-start_time:-start_time+1]

        X_train_2 = np.array(X_train_2)
        y_train = np.array(y_train)
        X_test_2 = np.array(X_test_2)
        y_test = np.array(y_test)
        X_test_2 = X_test_2.reshape(X_test_2.shape[0], X_test_2.shape[1], 1)

        # 重塑输入数据为 [samples, time steps, features] 格式
        X_train_2 = X_train_2.reshape(X_train_2.shape[0], X_train_2.shape[1], 1)
        # 构建LSTM模型
        model_2 = Sequential()
        model_2.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model_2.add(LSTM(50, return_sequences=False))
        model_2.add(Dense(25))
        model_2.add(Dense(1))

        # 编译模型
        model_2.compile(optimizer='adam', loss='mean_squared_error')
        model_2.fit(X_train_2, y_train, batch_size=32, epochs=20)
        train_predict_2 = model_2.predict(X_train_2)
        train_predict_2 = scaler.inverse_transform(train_predict_2)
        True_data_2 = scaler.inverse_transform(y_train.reshape(-1, 1))
        print(calculate_mape(train_predict_2, True_data_2))
        return_y_2 =  model_2.predict(X_test_2)
        return_y_pred_2 = scaler.inverse_transform(np.array(return_y_2).reshape(-1, 1))
        now_predict_2 = pd.DataFrame(return_y_pred_2,columns = ['predict'])
        now_predict_2['datetime'] = now_time_list[-start_time:-start_time+1]
        print(calculate_mape(y_test, return_y_pred_2))
        if calculate_mape(y_test, return_y_pred_1) < calculate_mape(y_test, return_y_pred_2):
            now_predict_1['day'] = 1
            now_predict_1['day'] = 1
            now_predict_1['day'] = 1

            all_predict = pd.concat([now_predict_1,all_predict])
            future_X = now_fill_na_predict[X_columns].to_numpy()
            future_data = scaler.inverse_transform(np.array(\
                            model_1.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
            future_data = pd.DataFrame(future_data,columns = ['predict'])
            all_feature = pd.concat([all_feature,future_data])
        else:
            all_predict = pd.concat([now_predict_2,all_predict])
            future_X = now_fill_na_predict[X_columns2].to_numpy()
            future_data = scaler.inverse_transform(np.array(\
                            model_2.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
            future_data = pd.DataFrame(future_data,columns = ['predict'])
            all_feature = pd.concat([all_feature,future_data])
    all_predict = pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
    all_predict = all_predict.sort_values('datetime').reset_index(drop = True)
    #all_predict.to_excel(r'return2.xlsx')



Epoch 1/20


C:\Users\nemo\anaconda3\envs\lstm_env\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 220ms/step - loss: 0.1590
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - loss: 0.0980
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - loss: 0.0735
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step - loss: 0.0546
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - loss: 0.0691
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - loss: 0.0647
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - loss: 0.0601
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step - loss: 0.0532
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step - loss: 0.0627
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step - loss: 0.0543
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step - loss: 0.0476
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - loss: 0.0581
Epoch 13/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - loss: 0.0591
Epoch 14/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step - loss: 0.0468
Epoch 15/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - loss: 0.0538
Epoch 16/20
2/2 ━━━━━━━━━━━━━━

C:\Users\nemo\anaconda3\envs\lstm_env\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 188ms/step - loss: 0.2671
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step - loss: 0.0861
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 248ms/step - loss: 0.1354
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step - loss: 0.0803
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - loss: 0.0547
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step - loss: 0.0768
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - loss: 0.0681
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step - loss: 0.0616
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - loss: 0.0501
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - loss: 0.0585
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - loss: 0.0614
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step - loss: 0.0636
Epoch 13/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - loss: 0.0541
Epoch 14/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step - loss: 0.0511
Epoch 15/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - loss: 0.0470
Epoch 16/20
2/2 ━━━━━━━━━━━━━━

...

C:\Users\nemo\anaconda3\envs\lstm_env\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 190ms/step - loss: 0.0894
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 186ms/step - loss: 0.0817
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 176ms/step - loss: 0.0584
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 179ms/step - loss: 0.0640
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 201ms/step - loss: 0.0471
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 200ms/step - loss: 0.0537
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 206ms/step - loss: 0.0535
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 193ms/step - loss: 0.0534
Epoch 9/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 188ms/step - loss: 0.0472
Epoch 10/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 205ms/step - loss: 0.0505
Epoch 11/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 185ms/step - loss: 0.0556
Epoch 12/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 186ms/step - loss: 0.0465
Epoch 13/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 190ms/step - loss: 0.0493
Epoch 14/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 186ms/step - loss: 0.0493
Epoch 15/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 203ms/step - loss: 0.0497
Epoch 16/20
3/3 ━━━━━━━━━━━━━━

C:\Users\nemo\anaconda3\envs\lstm_env\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 186ms/step - loss: 0.0430
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 191ms/step - loss: 0.0117
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 189ms/step - loss: 0.0093
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 209ms/step - loss: 0.0093
Epoch 5/20
13/16 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - loss: 0.0094

KeyboardInterrupt: 

In [ ]:
now_lstm_data[['offered']][-start_time:]

In [55]:
all_predict

,predict,datetime
0,5.110925,2025-03-20 08:30:00
0,1.617387,2025-02-21 08:00:00


In [56]:
all_feature

,predict
0,1.821418
0,4.333144


In [ ]:
all_predict['diff'] = (all_predict.predict - all_predict.offered)

In [ ]:
all_predict['hour'] = all_predict.datetime.dt.hour

In [ ]:
pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])

In [ ]:
calculate_mape(all_predict.offered,all_predict.predict)

In [ ]:
all_predict.sort_values('datetime')

In [36]:
all_predict.to_excel(r'return1-.xlsx')

In [40]:
data[add_column + ['offered']].corr()

,actans,actabn,absActHt,absActSa,offered
actans,1.000000,0.067562,0.891382,0.306075,0.881601
actabn,0.067562,1.000000,0.159377,0.632675,0.389850
absActHt,0.891382,0.159377,1.000000,0.400931,0.887095
absActSa,0.306075,0.632675,0.400931,1.000000,0.495375
offered,0.881601,0.389850,0.887095,0.495375,1.000000


In [47]:
all_feature

,predict
0,2.465809
1,3.281004
2,2.308758
3,2.679123
4,2.779515
...,...
53,13.024989
54,13.060239
55,13.140845
56,13.205191


In [41]:
all_predict = pd.concat([now_predict_1,all_predict])
future_X = now_fill_na_predict[X_columns].to_numpy()
future_data = scaler.inverse_transform(np.array(\
                model_1.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
future_data = pd.DataFrame(future_data,columns = ['predict'])
all_feature = pd.concat([all_feature,future_data])

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step


In [43]:
all_predict

,predict,datetime,offered
0,13.169405,2025-01-23 19:00:00,NaN
1,13.432196,2025-01-24 19:00:00,NaN
2,13.250965,2025-01-27 19:00:00,NaN
3,13.148877,2025-01-28 19:00:00,NaN
4,13.017861,2025-01-29 19:00:00,NaN
...,...,...,...
1329,44.976898,2025-04-15 17:00:00,37.0
1330,37.730900,2025-04-15 17:30:00,32.0
1331,33.240807,2025-04-15 18:00:00,20.0
1332,27.432621,2025-04-15 18:30:00,21.0


In [44]:
all_predict = pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])

In [45]:
all_predict.sort_values('datetime')

,predict,datetime,offered_x,offered_y
58,2.115029,2025-01-02 08:00:00,2.0,2.0
59,2.106689,2025-01-03 08:00:00,7.0,7.0
60,2.005751,2025-01-06 08:00:00,2.0,2.0
61,1.855358,2025-01-07 08:00:00,1.0,1.0
62,1.850668,2025-01-08 08:00:00,2.0,2.0
...,...,...,...,...
1388,37.730900,2025-04-15 17:30:00,32.0,32.0
1389,33.240807,2025-04-15 18:00:00,20.0,20.0
1390,27.432621,2025-04-15 18:30:00,21.0,21.0
57,13.227732,2025-04-15 19:00:00,NaN,12.0


In [51]:
all_feature = all_feature[:-58]

In [53]:
all_feature

,predict
0,2.465809
1,3.281004
2,2.308758
3,2.679123
4,2.779515
...,...
53,14.935564
54,14.947142
55,14.834311
56,14.826351


In [54]:
time_ = []
for (hour,minute) in fill_na_predict[['hour','minute']].drop_duplicates().to_numpy()[:]:
    now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
    time_.append(now_fill_na_predict.datetime.to_numpy())
all_feature['datetime'] =  np.hstack(time_)
all_feature.predict = all_feature.predict.map(lambda x:max(0,x))
all_feature.to_excel(r'feature2.xlsx')

C:\Users\nemo\AppData\Local\Temp\ipykernel_6016\1055264202.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_feature['datetime'] =  np.hstack(time_)
C:\Users\nemo\AppData\Local\Temp\ipykernel_6016\1055264202.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_feature.predict = all_feature.predict.map(lambda x:max(0,x))


In [56]:
all_predict

,predict,datetime,offered_x,offered_y
0,13.169405,2025-01-23 19:00:00,NaN,25.0
1,13.432196,2025-01-24 19:00:00,NaN,12.0
2,13.250965,2025-01-27 19:00:00,NaN,28.0
3,13.148877,2025-01-28 19:00:00,NaN,19.0
4,13.017861,2025-01-29 19:00:00,NaN,17.0
...,...,...,...,...
1387,44.976898,2025-04-15 17:00:00,37.0,37.0
1388,37.730900,2025-04-15 17:30:00,32.0,32.0
1389,33.240807,2025-04-15 18:00:00,20.0,20.0
1390,27.432621,2025-04-15 18:30:00,21.0,21.0


In [55]:
all_feature

,predict,datetime
0,2.465809,2025-04-11 08:00:00
1,3.281004,2025-04-14 08:00:00
2,2.308758,2025-04-15 08:00:00
3,2.679123,2025-04-16 08:00:00
4,2.779515,2025-04-17 08:00:00
...,...,...
53,14.935564,2025-06-27 19:00:00
54,14.947142,2025-06-30 19:00:00
55,14.834311,2025-07-01 19:00:00
56,14.826351,2025-07-02 19:00:00


In [ ]:
calculate_mape(all_predict.predict, all_predict.offered)

In [ ]:
(np.mean((all_predict.predict-all_predict.offered)**2))**0.5

In [ ]:
all_predict['hour'] = all_predict.datetime.dt.hour

In [ ]:
for hour in np.unique(all_predict.hour):
    _ = all_predict[all_predict.hour == hour]
    print(f'hour is {hour} the mape ：',calculate_mape(_.predict, _.offered))

In [ ]:
for hour in np.unique(all_predict.hour):
    _ = all_predict[all_predict.hour == hour]
    print(f'hour is {hour} the mse ：',(np.mean((_.predict-_.offered)**2))**0.5)

In [ ]:
1

In [ ]:
_ = all_predict[all_predict.hour == 11]

In [ ]:
calculate_mape(_.predict, _.offered)

In [ ]:
all_predict

In [ ]:
feature_[['predict','datetime']]

In [ ]:
all_predict = pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
all_predict = all_predict.sort_values('datetime').reset_index(drop = True)


In [ ]:
all_predict.to_excel(r'return.xlsx')

In [ ]:
datetime

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
data11 = lstm_data[lstm_data.hour == 11]

In [ ]:
data11.groupby('datetime').min().offered.sort_values()

In [ ]:

import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt

# 创建示例数据
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=all_predict['datetime'],  # 将 '日期' 列作为 X 轴
    y=all_predict['offered'],  # 将 '销售额' 列作为 Y 轴
    mode='lines+markers',  # 折线 + 数据点
    name='真实值'  # 图例名称
))
fig.add_trace(go.Scatter(
    x=all_predict['datetime'],  # 将 '日期' 列作为 X 轴
    y=all_predict['predict'],  # 将 '销售额' 列作为 Y 轴
    mode='lines+markers',  # 折线 + 数据点
    name='预测值'  # 图例名称
))

# 设置坐标轴名称
fig.update_layout(
    title='折线图示例',
    xaxis_title='X轴',
    yaxis_title='Y轴',
    template='plotly_white'
)
fig.show()

In [ ]:

# 可视化训练数据的预测结果
plt.figure(figsize=(10, 6))
plt.plot( all_predict['datetime'], all_predict['offered'] , label='Actual Data')
plt.plot( all_predict['datetime'], all_predict['predict'], label='Predicted Data')
plt.legend()
plt.show()
# 预测未来数据
future_steps = 30
future_predictions = []

last_sequence = X_train[-1]


In [ ]:
now_lstm_data

In [ ]:
now_lstm_data.offered.min()

In [ ]:
_ = lstm_data.groupby(['year','month','day']).offered.sum()
_.sort_values()[:30]

In [ ]:
now_lstm_data[X_columns]

In [ ]:

# 可视化训练数据的预测结果
plt.figure(figsize=(10, 6))
plt.plot(now_lstm_data[['offered']][1000:].to_numpy(), label='Actual Data')
plt.plot(return_y_pred, label='Predicted Data')
plt.legend()
plt.show()
# 预测未来数据
future_steps = 30
future_predictions = []

last_sequence = X_train[-1]


In [ ]:
predict_y =  model.predict( now_lstm_data[X_columns][:1000].to_numpy().reshape(-1, X_train.shape[1], 1))
predict_y = scaler.inverse_transform(np.array(predict_y).reshape(-1, 1))

In [ ]:
calculate_mape(now_lstm_data[['offered']][:1000].to_numpy(), predict_y)

In [ ]:
_ = ((np.abs((now_lstm_data[['offered']][1000:].to_numpy() - return_y_pred)/now_lstm_data[['offered']][1000:].to_numpy())))*100

In [ ]:
_.sort()

In [ ]:
np.argmax(_)

In [ ]:
_[298]

In [ ]:
now_lstm_data[['offered']][1000:].to_numpy()[298]